In [1]:
import pandas as pd
import math
import numpy as np
import graphviz 
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt

In [ ]:
"""cancertypes = ['BLCA','BRCA','CESC','CHOL', COAD','ESCA','GBM','HNSC','KICH',
                'KIRC','KIRP','LIHC','LUAD','LUSC','PAAD','PCPG','PRAD','READ',
                'SARC','SKCM','STAD','THCA','THYM','UCEC']"""

data_dir = "../Data"
cancertypes = ['BLCA','BRCA','CESC', 'COAD','GBM','HNSC','KICH',
               'KIRC','KIRP','LIHC','LUAD','LUSC','PRAD','READ',
               'SKCM','STAD','THCA','UCEC']
# cancertypes = ['BLCA','BRCA','CESC', 'COAD','GBM','HNSC','KICH']
# Ignore CHOL, ESCA, PAAD, PCPG, SARC, THYM

datatype = "FPKM" #  ["FPKM", "FeatureCounts", "TPM"]
tissue = "Tumor"
random_state = np.random.RandomState(0)

#### Create numerical data set for Random Forest

Question of interest to answer with this model: Can we predict tumor type based on gene expression?

In [ ]:
dataframe_dict = dict()
data = pd.DataFrame()
for cancertype in cancertypes:
    path = "/".join((data_dir, "CSV", "%s_%s_%s.csv" % (tissue, cancertype, datatype)))
    df = pd.read_csv(path, index_col=0)
    data = data.append(df)
print("Samples: %i\nGenes: %i" % data.shape)
data.head()

Partition traning and testing datasets

In [ ]:
target_map_dict = dict((i, cancertype) for i, cancertype in enumerate(cancertypes))
target_array = pd.factorize(data['Tumor Type'])[0]
train, test, target_array_training, target_array_testing = train_test_split(data, target_array, test_size=0.3)
gene_names = [val for val in data.columns.values if val != "Tumor Type"]
target_names = [v for v in target_map_dict.values()]

print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:',len(test))
print(target_map_dict)

#### Cross Validation & determination of optimal parameters

In [ ]:
pipeline = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(penalty="l2"))),
  ('classification', RandomForestClassifier(random_state=random_state))])



param_grid = {
    "classification__n_estimators": [10, 25, 50, 100], 
    "classification__max_depth": [5, 15],
    "classification__max_features": ["auto"], 
    "classification__criterion": ["gini"],
             }

cv = StratifiedKFold(n_splits=10, random_state=random_state)

grid = GridSearchCV(pipeline, cv=cv, param_grid=param_grid)
grid.fit(train[gene_names], target_array_training)

# summarize results
print("Best: %f using %s" % (grid.best_score_, 
    grid.best_params_))
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
max_depth = grid.best_params_["classification__max_depth"]
n_trees = grid.best_params_["classification__n_estimators"]
max_features = grid.best_params_["classification__max_features"]
criterion = grid.best_params_["classification__criterion"]

#### Select Features, Make and Train Random Forest using best params

In [ ]:
pipeline = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(penalty="l2"))),
  ('classification', RandomForestClassifier(n_estimators=n_trees, 
                                            criterion=criterion, 
                                            max_depth=max_depth, 
                                            max_features=max_features,
                                            random_state=random_state))])

clf = pipeline.fit(train[gene_names], target_array_training)
clf = clf.steps[1][1]

feature_names = []
for i in pipeline.named_steps["feature_selection"].get_support(True):
    feature_names.append(gene_names[i])

#### Predict and obtain confusion matrix

In [ ]:
predictions = clf.predict(test[feature_names])
predictions = np.array([target_map_dict[i] for i in predictions])
actual = test["Tumor Type"].values

confusion_matrix = pd.crosstab(actual, predictions, rownames=['Actual Tumor Type'], colnames=['Predicted Tumor Type'])
confusion_matrix.to_csv("/".join((data_dir, "CSV", "Confusion_maxdepth%i.csv" % max_depth)))
confusion_matrix

#### View Feature Importance, save trees

In [ ]:
feature_importance = list(zip(feature_names, clf.feature_importances_))
for feature in feature_importance.copy():
    if feature[1] == 0:
        feature_importance.remove(feature)
print("Number of importance features: %i" % len(feature_importance))
feature_importance = pd.DataFrame(feature_importance, columns=["Gene", "Relative Importance"])
feature_importance.set_index("Gene", inplace=True)
feature_importance.sort_values(by=['Relative Importance'], ascending=False, inplace=True)
feature_importance.to_csv("/".join((data_dir, "CSV", "FeatureImportance_maxdepth%i.csv" % max_depth)))
feature_importance

In [ ]:
for i, estimator in enumerate(clf.estimators_):
    dot_data = tree.export_graphviz(estimator, out_file=None, 
                             feature_names=feature_names,  
                             class_names=target_names,  
                             filled=True, rounded=True,  
                             special_characters=True)  
    graph = graphviz.Source(dot_data)  
    with open("/".join(("../Images", "Trees/Decision Tree %i.svg")) % i, 'w') as f:
        f.write(graph._repr_svg_())
    if i == 1:
        break
graph

#### Performance Evaluation

In [ ]:
n_classes = len(cancertypes)
target_array_testing = label_binarize(target_array_testing, classes=[i for i in range(0, n_classes)])
probs = clf.predict_proba(test[feature_names])

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(0, n_classes):
    fpr[i], tpr[i], _ = roc_curve(target_array_testing[:, i], probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    plt.plot(fpr[i], tpr[i],
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(target_map_dict[i], roc_auc[i]))
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', alpha=.8)
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
title_str = 'ROC Max Depth %i.jpeg' % max_depth
plt.title(title_str)
plt.legend(loc="center right", bbox_to_anchor=(1.75, 0.5))
plt.show()